###### The cell below loads the visual style of the notebook when run.

In [0]:
from IPython.core.display import HTML
css_file = '../../styles/styles.css'
HTML(open(css_file, "r").read())

# Fitting Polynomials using NumPy

Let's load the data from the main notebook and plot it.

In [0]:
%matplotlib inline  
import numpy as np
import matplotlib.pyplot as plt

x,y,e = np.loadtxt('/home/user/PHY241/data/Session7/data.txt',unpack=True)
fig,ax = plt.subplots()
ax.errorbar(x,y,yerr=e,fmt='.')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

----

### Fitting polynomials with ```numpy.polyfit```

The data looks like it is well described by a straight line. The easiest way to fit a polynomial like a straight line, or a quadratic function is to use ```numpy```'s in-built polynomial fitting routines. First of all we'll use ```numpy``` in the most basic form to fit the x and y data, ignoring the uncertainties.

In [0]:
degree = 1 # order of the polynomial to fit.

"""np.polyfit takes three arguments - the x data, the y data 
   and the order of the polynomial. 
   
   it returns a list of polynomial parameters, with the highest
   order first. For example, if you fit y = a1*x + a0 the result
   is returned as [a1,a0].
"""
poly_params = np.polyfit(x,y,degree)
print( poly_params )

Note two things: that the polyfit routine returns a list of the polynomial parameters, and that numpy refers to a straight line as a polynomial of order 1. In this case, the list is [gradient, intercept].

Also notice that we don't supply the uncertainties on our data to ```polyfit``` - in this case, polyfit is minimising the sum-of-the-squares, ${\rm SS}$.

Now we have our fit, let's plot it on top of the data. First, we create a polynomial function from our best-fit parameters

In [0]:
poly = np.poly1d(poly_params)

The line above creates a function, which we store with the name *poly*. We can calculate the value of this function at our data points, *x*, to create a new array of our best fit values:

In [0]:
yfit = poly(x)

Now let's plot our best fit values and our data together:

In [0]:
fig,ax = plt.subplots()
# plot our data as errorbars
ax.errorbar(x,y,yerr=e,fmt='.')
# plot our best fit as a line
ax.plot(x,yfit)
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

### Fitting $\chi^2$

What if we don't want to ignore the errors? We can use them as <b>weights</b> in the fit, and so get ```polyfit``` to minimise $\chi^2$. To do this, we want to weight each point by the inverse of the variance.

In [0]:
weights = 1./e**2
poly_params = np.polyfit(x,y,degree,w=weights)
print(poly_params)

What if we want to know the uncertainties on our best fit? This will be true most of the time! We can ask the polyfit routine to return the <em>covariance matrix</em> for our parameters. You'll learn more about covariance matrices next year, for now you can assume that the diagonal elements of this matrix are our uncertainties, and these can easily be extracted using numpy's <code>diag</code> function

In [0]:
poly_params, covar_mat = np.polyfit(x,y,degree,w=weights,cov=True)
print(covar_mat)

In [0]:
param_errs = np.diag(covar_mat)
print(param_errs)

Finally, we can use string formatting to print out model fit and errors neatly and to the correct number of significant figures. For a reminder about string formatting - see [here](http://openbookproject.net/thinkcs/python/english3e/strings.html#the-string-format-method).

In [0]:
gradient, intercept = poly_params
gradient_err, intercept_err = param_errs

print("The gradient is {:.2f} +/- {:.2f}".format(gradient, gradient_err))
print("The intercept is {:.0f} +/- {:.0f}".format(intercept, intercept_err))